Objective: Make a selection paintings for the exhibition catalogue to be selected from Wikidata and rendered multi-format in Quarto.

The below Python code uses SPARQLWrapper to retrieve data from Wikidata based on a SPARQL query.

In [4]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

# query URL https://query.wikidata.org/#SELECT%20%0A%20%20%3Fpainting%20%0A%20%20%3FcreatorLabel%0A%20%20%3Fimage%0A%20%20%3Fcopyright_status%20%0A%20%20%3Finventory_number%20%0A%20%20%28GROUP_CONCAT%28DISTINCT%20%3Fmade_from_materialLabel%3B%20separator%3D%22%2C%20%22%29%20AS%20%3Fmade_from_materialLabels%29%20%0A%20%20%3FlocationLabel%0A%20%20%28GROUP_CONCAT%28DISTINCT%20%3FgenreLabel%3B%20separator%3D%22%2C%20%22%29%20AS%20%3FgenreLabels%29%20%0A%20%20%28GROUP_CONCAT%28DISTINCT%20%3FdepictsLabel%3B%20separator%3D%22%2C%20%22%29%20AS%20%3FdepictsLabels%29%20%0A%20%20%3Fmain_subjectLabel%0AWHERE%20%7B%0A%20%20%3Fpainting%20wdt%3AP31%20wd%3AQ3305213%3B%20%20%23%20Instance%20of%20%22painting%22%0A%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP195%20wd%3AQ160236%3B%20%20%20%23%20Part%20of%20the%20Metropolitan%20Museum%20of%20Art%20collection%0A%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP180%20wd%3AQ4022.%20%20%20%20%23%20Depicts%20%27river%27%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP170%20%3Fcreator.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP18%20%3Fimage.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP6216%20%3Fcopyright_status.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP571%20%3Finception.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP217%20%3Finventory_number.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP186%20%3Fmade_from_material.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP276%20%3Flocation.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP136%20%3Fgenre.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP180%20%3Fdepicts.%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fpainting%20wdt%3AP921%20%3Fmain_subject.%20%7D%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%0A%20%20%20%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%0A%20%20%20%20%3Fcreator%20rdfs%3Alabel%20%3FcreatorLabel.%0A%20%20%20%20%3Fmade_from_material%20rdfs%3Alabel%20%3Fmade_from_materialLabel.%0A%20%20%20%20%3Flocation%20rdfs%3Alabel%20%3FlocationLabel.%0A%20%20%20%20%3Fgenre%20rdfs%3Alabel%20%3FgenreLabel.%0A%20%20%20%20%3Fdepicts%20rdfs%3Alabel%20%3FdepictsLabel.%0A%20%20%20%20%3Fmain_subject%20rdfs%3Alabel%20%3Fmain_subjectLabel.%0A%20%20%7D%0A%7D%0AGROUP%20BY%20%3Fpainting%20%3FcreatorLabel%20%3Fimage%20%3Fcopyright_status%20%3Finventory_number%20%3Fmade_from_materialLabels%20%3FlocationLabel%20%3FgenreLabels%20%3FdepictsLabels%20%3Fmain_subjectLabel%0AORDER%20BY%20%3Fpainting%0ALIMIT%2027%0A

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT 
  ?painting 
  ?creatorLabel
  ?image
  ?copyright_status 
  ?inventory_number 
  (GROUP_CONCAT(DISTINCT ?made_from_materialLabel; separator=", ") AS ?made_from_materialLabels) 
  ?locationLabel
  (GROUP_CONCAT(DISTINCT ?genreLabel; separator=", ") AS ?genreLabels) 
  (GROUP_CONCAT(DISTINCT ?depictsLabel; separator=", ") AS ?depictsLabels) 
  ?main_subjectLabel
WHERE {
  ?painting wdt:P31 wd:Q3305213;  # Instance of "painting"
            wdt:P195 wd:Q160236;   # Part of the Metropolitan Museum of Art collection
            wdt:P180 wd:Q4022.    # Depicts 'river'
  OPTIONAL { ?painting wdt:P170 ?creator. }
  OPTIONAL { ?painting wdt:P18 ?image. }
  OPTIONAL { ?painting wdt:P6216 ?copyright_status. }
  OPTIONAL { ?painting wdt:P571 ?inception. }
  OPTIONAL { ?painting wdt:P217 ?inventory_number. }
  OPTIONAL { ?painting wdt:P186 ?made_from_material. }
  OPTIONAL { ?painting wdt:P276 ?location. }
  OPTIONAL { ?painting wdt:P136 ?genre. }
  OPTIONAL { ?painting wdt:P180 ?depicts. }
  OPTIONAL { ?painting wdt:P921 ?main_subject. }

  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
    ?creator rdfs:label ?creatorLabel.
    ?made_from_material rdfs:label ?made_from_materialLabel.
    ?location rdfs:label ?locationLabel.
    ?genre rdfs:label ?genreLabel.
    ?depicts rdfs:label ?depictsLabel.
    ?main_subject rdfs:label ?main_subjectLabel.
  }
}
GROUP BY ?painting ?creatorLabel ?image ?copyright_status ?inventory_number ?made_from_materialLabels ?locationLabel ?genreLabels ?depictsLabels ?main_subjectLabel
ORDER BY ?painting
LIMIT 27
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'painting': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q104412992'}, 'creatorLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'William Guy Wall'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/New%20York%20from%20the%20Heights%20near%20Brooklyn%20MET%20DT5423.jpg'}, 'copyright_status': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19652'}, 'inventory_number': {'type': 'literal', 'value': '54.90.301'}, 'made_from_materialLabels': {'type': 'literal', 'value': ''}, 'locationLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Metropolitan Museum of Art'}, 'genreLabels': {'type': 'literal', 'value': 'landscape art'}, 'depictsLabels': {'type': 'literal', 'value': 'river, boat, landscape art'}}
{'painting': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q116444817'}, 'made_from_materialLabels': {'type': 'literal', 'value': ''}, 'genreLabels': {'type': 'literal', 'value': 'landscape art'}, 'depictsLabels': {